In [ ]:
import json
import zipfile
from pathlib import Path
import time

DATA_PATH = "/kaggle/input/google-code-golf-2025"  # Update if needed

# Simplified pattern detectors (pure Python, no NumPy for code golf)
def detect_pattern(inp, out):
    """Detect simple patterns; return (pattern_name, param) or None"""
    # Identity
    if inp == out:
        return ('IDENTITY', None)
    
    # Flip horizontal
    if [row[::-1] for row in inp] == out:
        return ('FLIP_H', None)
    
    # Flip vertical
    if inp[::-1] == out:
        return ('FLIP_V', None)
    
    # Rotate 90 CW (transpose reverse rows)
    if list(map(list, zip(*inp[::-1]))) == out:
        return ('ROT_90', None)
    
    # Color replace (simple map)
    flat_in = [c for row in inp for c in row]
    flat_out = [c for row in out for c in row]
    if len(set(flat_in)) == len(set(flat_out)):
        cmap = dict(zip(set(flat_in), set(flat_out)))
        if all(cmap.get(c, c) == flat_out[i] for i, c in enumerate(flat_in)):
            return ('COLOR_REPLACE', cmap)
    
    # Fill background with color
    zeros_in = [(i,j) for i,row in enumerate(inp) for j,c in enumerate(row) if c==0]
    if zeros_in and all(out[i][j] == out[zeros_in[0][0]][zeros_in[0][1]] for i,j in zeros_in):
        fill_c = out[zeros_in[0][0]][zeros_in[0][1]]
        if all(inp[i][j] == out[i][j] for i,row in enumerate(inp) for j,c in enumerate(row) if c!=0):
            return ('FILL_BG', fill_c)
    
    # Gravity down (per column)
    h, w = len(inp), len(inp[0]) if inp else 0
    expected = [[0]*w for _ in range(h)]
    for c in range(w):
        col_nonzero = [inp[r][c] for r in range(h) if inp[r][c] != 0]
        for i in range(len(col_nonzero)):
            expected[h-len(col_nonzero)+i][c] = col_nonzero[i]
    if expected == out:
        return ('GRAVITY', None)
    
    # Majority color fill
    flat = [c for row in inp for c in row if c != 0]
    if flat:
        maj = Counter(flat).most_common(1)[0][0]
        if all(row == [maj]*len(row) for row in out):
            return ('MAJORITY', maj)
    
    # Extract color
    for col in range(10):
        expected = [[c if c==col else 0 for c in row] for row in inp]
        if expected == out:
            return ('EXTRACT', col)
    
    # Repeat row
    if len(inp) % len(out) == 0 and all(out[i] == inp[i % len(inp)] for i in range(len(out))):
        return ('REPEAT_ROW', len(inp))
    
    return None

def generate_short_code(pattern, param):
    """Generate shortest Python function for pattern."""
    if pattern == 'IDENTITY':
        return "def solve(t):return t['test'][0]['input']"
    
    if pattern == 'FLIP_H':
        return "def solve(t):return[[c[::-1]for c in r]for r in t['test'][0]['input']]"
    
    if pattern == 'FLIP_V':
        return "def solve(t):return t['test'][0]['input'][::-1]"
    
    if pattern == 'ROT_90':
        return "def solve(t):return[list(r)for r in zip(*t['test'][0]['input'][::-1])]"
    
    if pattern == 'COLOR_REPLACE':
        # Short map: assume small dict
        m = param
        map_str = ''.join(f"if c=={k}else{v}" for k,v in m.items()) + 'else c'
        return f"def solve(t):return[[{map_str}for c in r]for r in t['test'][0]['input']]"
    
    if pattern == 'FILL_BG':
        fill = param
        return f"def solve(t):return[[c if c else{fill}else c for c in r]for r in t['test'][0]['input']]"
    
    if pattern == 'GRAVITY':
        return "def solve(t):return[list(c)for c in zip(*[[[0]*(len(t['test'][0]['input'])-len([r for r in col if r]))+[r for r in col if r]for col in zip(*t['test'][0]['input'])]])]"
    
    if pattern == 'MAJORITY':
        maj = param
        return f"def solve(t):return[[{maj}]*len(r)for r in t['test'][0]['input']]"
    
    if pattern == 'EXTRACT':
        col = param
        return f"def solve(t):return[[c if c=={col}else 0 for c in r]for r in t['test'][0]['input']]"
    
    if pattern == 'REPEAT_ROW':
        n = param
        return f"def solve(t):return sum([[r]*{n}for r in t['test'][0]['input']],[])"
    
    return "def solve(t):return t['test'][0]['input']"  # Fallback

# Generate submission ZIP
def generate_submission():
    data_path = Path(DATA_PATH)
    task_files = sorted(data_path.glob('*.json'))  # Assume tasks are JSON files
    
    print(f"Processing {len(task_files)} tasks...")
    
    solved = 0
    with zipfile.ZipFile('submission.zip', 'w') as zf:
        for i, task_file in enumerate(task_files):
            task_id = task_file.stem
            try:
                task = json.loads(task_file.read_text())
                
                # Detect pattern from first train example
                ex = task['train'][0]
                pattern = detect_pattern(ex['input'], ex['output'])
                
                if pattern:
                    code = generate_short_code(pattern[0], pattern[1])
                    solved += 1
                else:
                    code = "def solve(t):return t['test'][0]['input']"
                
                # Write to file
                filename = f"task{task_id.zfill(3)}.py"
                zf.writestr(filename, code)
                
                if (i + 1) % 50 == 0:
                    print(f"Progress: {i+1}/{len(task_files)} (Solved so far: {solved})")
                    
            except Exception as e:
                print(f"Error on {task_id}: {e}")
                # Fallback
                code = "def solve(t):return t['test'][0]['input']"
                filename = f"task{task_id.zfill(3)}.py"
                zf.writestr(filename, code)
    
    print(f"\nGenerated submission.zip with {solved}/{len(task_files)} solved patterns.")
    print("Upload to Kaggle: Go to competition → Submit → Upload submission.zip")

if __name__ == "__main__":
    generate_submission()